In [1]:
import sys
import os
sys.path.append('/Users/jamesmaskill/Desktop/Decision_Transformers/VS_Implementation/src')
sys.path.append('/Users/jamesmaskill/Desktop/Decision_Transformers/VS_Implementation/scripts')
from drawdown_control.decision_transformer import *
from drawdown_control.decision_transformer_policy import *
from drawdown_control.trajectory_processing import *
from drawdown_control.evaluate import *
from UtilsRL.logger import CompositeLogger
from torch.utils.data import DataLoader
from UtilsRL.exp import parse_args, setup
import d4rl
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import pandas as pd
import plotly.express as px
initial = set(dir())

No module named 'flow'
No module named 'dm_control.mujoco'
No module named 'carla'
pybullet build time: Jul 22 2024 18:55:30


In [2]:
from get_all_datasets import * 
current = set(dir())
datasets = [i for i in current - initial if i.endswith('dataset')]
print(datasets)

/Users/jamesmaskill/miniforge3/envs/py10env/lib/python3.10/site-packages/gym/envs/registration.py:511: UserWarning: WARN: Using the latest versioned environment `walker2d-expert-v2` instead of the unversioned environment `walker2d-expert`
  logger.warn(
/Users/jamesmaskill/miniforge3/envs/py10env/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|██████████| 21/21 [00:03<00:00,  6.59it/s]
/Users/jamesmaskill/miniforge3/envs/py10env/lib/python3.10/site-packages/gym/envs/registration.py:511: UserWarning: WARN: Using the latest versioned environment `walker2d-full-replay-v2` instead of the unversioned environment `walker2d-full-replay`
  logger.warn(
load datafile: 100%|██████████| 11/11 [00:03<00:00,  3.21it/s]
/Users/jamesmaskill/miniforge3/envs/py10env/lib/python3.10/site-packages/gym/envs/registration.py:511: UserWarning: WAR

['walker2d_medium_dataset', 'walker2d_expert_dataset', 'walker2d_full_replay_dataset', 'walker2d_medium_expert_dataset', 'walker2d_medium_replay_dataset', 'walker2d_random_dataset']


In [3]:
random = walker2d_random_dataset
medium_replay = walker2d_medium_replay_dataset
full_replay = walker2d_full_replay_dataset
medium_expert = walker2d_medium_expert_dataset
expert = walker2d_expert_dataset #bad
medium = walker2d_medium_dataset

In [ ]:
locations = [i for i, v in enumerate(medium_expert['terminals']) if v]
for l in locations:
    if l:
        print(f"{l}")

In [14]:
def get_rewards_and_drawdown(dataset):
    df = pd.DataFrame({
    'terminals': dataset['terminals'],
    'rewards': dataset['rewards'],
    'drawdowns': dataset['drawdowns']})
    df['episode'] = df['terminals'].cumsum()
    episodes = df.groupby('episode').apply(
        lambda x: pd.Series({'Rewards': x['rewards'].sum(),'Maximum Drawdown': x['drawdowns'].min()}), include_groups=False).reset_index()
    return list(zip(episodes['Rewards'], episodes['Maximum Drawdown']))
    

def get_rewards_and_timesteps(dataset):
    df = pd.DataFrame({
    'terminals': dataset['terminals'],
    'rewards': dataset['rewards']})
    df['episode'] = df['terminals'].cumsum()
    episodes = df.groupby('episode').apply(
        lambda x: pd.Series({'Timesteps': int(len(x)),'Rewards': round(x['rewards'].sum(),0)}), include_groups=False).reset_index()
    return list(zip(episodes['Rewards'], episodes['Timesteps']))


def plot_rewards_and_timesteps(datasets, labels): 
    all_rewards = []
    all_timesteps = []
    all_labels = []
    for dataset, label in zip(datasets, labels):
        rewards, timesteps = zip(*get_rewards_and_timesteps(dataset))
        all_rewards.extend(rewards)
        all_timesteps.extend(timesteps)
        all_labels.extend([label] * len(rewards)) 
    fig = px.scatter(x=all_rewards, y=all_timesteps, color=all_labels,
                     title="Distribution of Episode Rewards and Total Timesteps",
                     labels={'x': 'Sum of Rewards', 'y': 'Total Timesteps', 'color': 'Dataset'})
    fig.show()

def plot_rewards_and_drawdowns(datasets, labels): 
    all_rewards = []
    all_drawdowns = []
    all_labels = []
    for dataset, label in zip(datasets, labels):
        rewards, drawdowns = zip(*get_rewards_and_drawdown(dataset))
        all_rewards.extend(rewards)
        all_drawdowns.extend(drawdowns)
        all_labels.extend([label] * len(rewards)) 
    fig = px.scatter(x=all_rewards, y=all_drawdowns, color=all_labels,
                     title="Distribution of Episode Rewards and Maximum Drawdowns",
                     labels={'x': 'Sum of Rewards', 'y': 'Maximum Drawdowns', 'color': 'Dataset'})
    fig.show()
    
plot_rewards_and_drawdowns([random, medium_replay, full_replay,\
                            medium_expert, medium],
                            ["Random", "Medium Replay", "Full Replay",\
                             "Medium/Expert", "Medium"])


In [17]:
# Some code that gives us the 2D returns against drawdowns representations of the datasets 
# Some more code that gives us the 2D returns against episode lengths representations of the datasets

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (4039163215.py, line 1)

In [ ]:
# Some code that evals the decision transformer on the possibility space of the pairs of arguments and plots it as a scatter chart of hits and misses. 
# We can do the visualizatins even better by having a key of white (exact), yellow (within 10%), red (withiin 20%), black (not even close). 
# This is done as just a comparison between the parameters in a grid and the outcomes of the eval decisin transformer. 

In [ ]:
def eval_decision_transformer(env, policy, target_returns, return_scale, eval_episodes, seed, max_drawdowns):
    seed_all(seed)
    rewards = []
    lengths = []
    max_drawdown = []
    
    for _ in range(eval_episodes):
        state = env.reset()
        episode_return = 0
        episode_length = 0
        episode_max_drawdown = 0
        done = False
        while not done:
            action = policy.select_action(
                states=np.array([state]),
                actions=np.array([env.action_space.sample()]),
                returns_to_go=np.array([target_returns[0] * return_scale]),
                timesteps=np.array([episode_length]),
                max_drawdowns=np.array([max_drawdowns])
            )
            state, reward, done, _ = env.step(action)
            episode_max_drawdown = min(episode_max_drawdown, episode_max_drawdown+reward)
            episode_return += reward
            episode_length += 1
        rewards.append(episode_return)
        lengths.append(episode_length)
        max_drawdown.append(episode_max_drawdown)
    
    avg_reward = np.mean(rewards)
    avg_drawdown = np.mean(max_drawdown)
    
    return {"reward": avg_reward, "max_drawdown": avg_drawdown}

In [2]:
# Sample DataFrame
data = {
    'episode': [1, 2, 3, 4],
    'episode_length': [5, 7, 6, 4],
    'sum_rewards': [2.0, 1.5, 3.1, 0.8],
    'max_drawdown': [0.2, 0.5, 0.3, 0.4]
}

df = pd.DataFrame(data)

# Create an interactive scatter plot
fig = px.scatter(df, x='sum_rewards', y='max_drawdown', 
                 title="Sum of Rewards vs. Maximum Drawdown",
                 labels={'sum_rewards': 'Sum of Rewards', 'max_drawdown': 'Maximum Drawdown'})

# Show plot
fig.show()


ModuleNotFoundError: No module named 'plotly'